### Create a file that contains a list of file paths which store the graph files

In [19]:
file_path = "../Datasets/Plant Pollinator/"

#### Create `edge lists` from `adj. matrices`

In [20]:
# This module is imported so that we can use listdir or other function
import os 

In [22]:
'''To append the file path and file name that needs to be included in the adjacency matrix list. 
Endswith just indicates which files to include and the one with "Dryad" should be excluded.
'''
adj_matrices_list = [file_path+f for f in os.listdir(file_path) \
                     if f.endswith('Links.csv') and 'Dryad' not in f]
adj_matrices_list

['../Datasets/Plant Pollinator/Site10_TenerifeTeno2Links.csv',
 '../Datasets/Plant Pollinator/Site11_Gomera1Links.csv',
 '../Datasets/Plant Pollinator/Site12_Gomera2Links.csv',
 '../Datasets/Plant Pollinator/Site13_Hierro1Links.csv',
 '../Datasets/Plant Pollinator/Site14_Hierro2Links.csv',
 '../Datasets/Plant Pollinator/Site1_WesternSahara1Links.csv',
 '../Datasets/Plant Pollinator/Site2_WesternSahara2Links.csv',
 '../Datasets/Plant Pollinator/Site3_Fuerteventura1Links.csv',
 '../Datasets/Plant Pollinator/Site4_Fuerteventura2Links.csv',
 '../Datasets/Plant Pollinator/Site5_GranCanaria1Links.csv',
 '../Datasets/Plant Pollinator/Site6_GranCanaria2Links.csv',
 '../Datasets/Plant Pollinator/Site7_TenerifeSouth1Links.csv',
 '../Datasets/Plant Pollinator/Site8_TenerifeSouth2Links.csv',
 '../Datasets/Plant Pollinator/Site9_TenerifeTeno1Links.csv']

In [11]:
# Importing modules that are needed to run multired.py
import networkx as nx
import pandas as pd
import numpy as np
import scipy as sp
import csv

In [7]:
'''
Reading the csv file. Since the first column is also the node name therefore "index_col=0" tells panda to 
make the first column as the row name
'''
df_matrices = [pd.read_csv(adj_mat, index_col=0) for adj_mat in adj_matrices_list]

In [8]:
# Iterating through each matrix
for index in range(len(df_matrices)):
    mat = df_matrices[index] 
# Building a sparse matrix for networkx because networkx requires it for bipartite
    A = sp.sparse.csc_matrix(mat)
    g = nx.algorithms.bipartite.matrix.from_biadjacency_matrix(A)
    #bi_dict = nx.get_node_attributes(g, 'bipartite')
    #nx.set_node_attributes(g, 'bipartite', {k: 'pollinator' if bi_dict[k] == 0 \
    #                                        else 'plant' for k in bi_dict})
    #labels_list = mat.index.tolist() + mat.columns.tolist()
    #nx.relabel_nodes(g, {k: labels_list[k] for k in list(g)}, False)
# The above code is commented because it is not needed since we don't need the labels.

# Saving the edgelist to a csv file.
    with open('./%s_edge_list.csv'%adj_matrices_list[index].split('/')[-1][:-4], 'w') as edge_list_f:
        csv.writer(edge_list_f, delimiter=' ').writerows(g.edges())

In [15]:
# Creating a csv that will be used by multired

edge_list_file = [f for f in os.listdir('./') if f.endswith('edge_list.csv')]
edge_list_file

['Site10_TenerifeTeno2Links_edge_list.csv',
 'Site11_Gomera1Links_edge_list.csv',
 'Site12_Gomera2Links_edge_list.csv',
 'Site13_Hierro1Links_edge_list.csv',
 'Site14_Hierro2Links_edge_list.csv',
 'Site1_WesternSahara1Links_edge_list.csv',
 'Site2_WesternSahara2Links_edge_list.csv',
 'Site3_Fuerteventura1Links_edge_list.csv',
 'Site4_Fuerteventura2Links_edge_list.csv',
 'Site5_GranCanaria1Links_edge_list.csv',
 'Site6_GranCanaria2Links_edge_list.csv',
 'Site7_TenerifeSouth1Links_edge_list.csv',
 'Site8_TenerifeSouth2Links_edge_list.csv',
 'Site9_TenerifeTeno1Links_edge_list.csv']

In [10]:
# Creating a csv that will be used by multired (contd). Here we are actually writing to a file.
with open('edge_list_file.csv', 'w') as f:
    csv.writer(f).writerows([[item] for item in edge_list_file])

Use `multired.py` 

In [1]:
import multired as mr

In [2]:
# Instructions in github

multiplex_red = mr.multiplex_red('./edge_list_file.csv')

In [6]:
# Function to calculate Jensen Shannon Divergence

multiplex_red.compute_JSD_matrix()

In [16]:
df = pd.DataFrame(multiplex_red.JSD)
df.index = edge_list_file
df.columns = edge_list_file
df

,Site10_TenerifeTeno2Links_edge_list.csv,Site11_Gomera1Links_edge_list.csv,Site12_Gomera2Links_edge_list.csv,Site13_Hierro1Links_edge_list.csv,Site14_Hierro2Links_edge_list.csv,Site1_WesternSahara1Links_edge_list.csv,Site2_WesternSahara2Links_edge_list.csv,Site3_Fuerteventura1Links_edge_list.csv,Site4_Fuerteventura2Links_edge_list.csv,Site5_GranCanaria1Links_edge_list.csv,Site6_GranCanaria2Links_edge_list.csv,Site7_TenerifeSouth1Links_edge_list.csv,Site8_TenerifeSouth2Links_edge_list.csv,Site9_TenerifeTeno1Links_edge_list.csv
Site10_TenerifeTeno2Links_edge_list.csv,0.000000,0.590699,0.519793,0.639987,0.591140,0.590462,0.491619,0.626383,0.600921,0.598293,0.605189,0.524505,0.485093,0.530363
Site11_Gomera1Links_edge_list.csv,0.590699,0.000000,0.517724,0.460329,0.452112,0.507886,0.575880,0.470009,0.466269,0.397371,0.449261,0.503524,0.641174,0.409938
Site12_Gomera2Links_edge_list.csv,0.519793,0.517724,0.000000,0.597512,0.467807,0.460360,0.446763,0.497798,0.487885,0.485034,0.507013,0.364765,0.579748,0.501190
Site13_Hierro1Links_edge_list.csv,0.639987,0.460329,0.597512,0.000000,0.509292,0.498605,0.593998,0.537677,0.519296,0.412874,0.437607,0.604580,0.654596,0.453626
Site14_Hierro2Links_edge_list.csv,0.591140,0.452112,0.467807,0.509292,0.000000,0.415412,0.567928,0.404761,0.435770,0.428165,0.467483,0.489628,0.628555,0.461228
Site1_WesternSahara1Links_edge_list.csv,0.590462,0.507886,0.460360,0.498605,0.415412,0.000000,0.537119,0.447767,0.365895,0.464792,0.409617,0.491837,0.624082,0.446055
Site2_WesternSahara2Links_edge_list.csv,0.491619,0.575880,0.446763,0.593998,0.567928,0.537119,0.000000,0.568306,0.571110,0.567213,0.567704,0.452556,0.598290,0.505003
Site3_Fuerteventura1Links_edge_list.csv,0.626383,0.470009,0.497798,0.537677,0.404761,0.447767,0.568306,0.000000,0.378702,0.448594,0.448391,0.507412,0.658758,0.474943
Site4_Fuerteventura2Links_edge_list.csv,0.600921,0.466269,0.487885,0.519296,0.435770,0.365895,0.571110,0.378702,0.000000,0.448350,0.384717,0.513752,0.649566,0.448309
Site5_GranCanaria1Links_edge_list.csv,0.598293,0.397371,0.485034,0.412874,0.428165,0.464792,0.567213,0.448594,0.448350,0.000000,0.361327,0.500087,0.615071,0.422767


In [17]:
df.to_csv('./JSD_06272017.csv')